In [ ]:
'''
1.The process of achieving specific predictions
2.An example of a prediction is provided for train 1S41.
'''

In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from xgboost import XGBClassifier as XGBC
from sklearn.externals import joblib
from sklearn import preprocessing

In [29]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['installbackupandsync.exe',
 'sample_submission.csv',
 'test_clean_kalman.csv',
 'train_clean_kalman.csv',
 'open-data-fin.csv',
 'Colab Notebooks',
 'Andrew_tensorflow_leeson',
 'newtrain.csv',
 'newtrain2.csv',
 'newtrain3.csv',
 'datawithpr_withtrainid.csv',
 'datawithpr_havepr.csv',
 'datapr1.0.csv',
 'datapr_planC.csv',
 'planC_with_state.csv',
 'full_data_with_state.csv',
 'T1K11_2.csv',
 'save',
 'Model',
 'LSTM',
 'lstm_model.h5',
 'new_clear_data.csv',
 'pdfullindex.csv',
 'datawithstate.csv',
 '120000_pd_pr_train_id.csv',
 '150000_pd_pr_train_id.csv',
 '180000_pd_pr_train_id.csv',
 '210000_pd_pr_train_id.csv',
 '240000_pd_pr_train_id.csv',
 '270000_pd_pr_train_id.csv',
 '300000_pd_pr_train_id.csv',
 'pd_full_pr_train_id.csv',
 'newderby_datawithca0.csv',
 'lstm_fullpred_model.h5',
 'new_testdata_with_ca.csv',
 'full_testdata_with_ca_state.csv',
 'open-data-2-derby.csv',
 '1M00_softmax.h5',
 '1S41_simpleRNN.h5',
 'lstm_fullpred_model_vs.h5',
 'simpleRNN_fullpred_model_vs.h5']

In [ ]:
'''
1.The process of achieving specific predictions
'''

In [ ]:
#LSTM pre-processing

In [30]:
def get_dict_(data,train_ID):
    data1=data[data['Train_ID']==train_ID]
    timestamp=[]
    for i in data1['Timestamp'].tolist():
        timestamp.append(i[0:2])
    data1['Timestamp']=timestamp


    data1=data1.iloc[:,0:4]
    data1.drop('ID',inplace=True,axis=1)
    data1.index= range(0,len(data1))

    event2=data1['Event2'].tolist()
    new_event=[]
    for i in event2:
        if i[0:3]=='RDY':
            new_event.append('RDY0'+i[3:6])
        elif i[0]=='R':
            new_event.append(i[0:7])
        else:
            new_event.append(i)
    data1['Event2']=new_event
    data1 = data1[data1['Type'].isin(['CA','Panel Request'])]
    all_sentence=data1['Event2'].tolist()
    allevent=data1['Event2'].unique().tolist()

    idx2char = np.array(allevent)
    char2idx={char:idx for idx,char in enumerate(allevent)}

    index_record=data1.index

    alls_with_index=[]
    each=[]
    for j in range(0,len(data1)):
        each=[all_sentence[j],index_record[j]]
        alls_with_index.append(each)

    length=len(data1)

    return allevent,new_event,alls_with_index,length

In [ ]:
# Predictive function

In [31]:
def pred_pr(model, pre_message,char2idx,idx2char,num_generate = 1):
    input_eval = [char2idx[ch] for ch in pre_message]
    input_eval = tf.expand_dims(input_eval, 0)
    
    pred_pr = []
    model.reset_states()

    temperature = 0.5
    
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions / temperature
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()
        pred_pr.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)
    return pred_pr

In [ ]:
# Get LSTM predictions results

In [32]:
def get_lstm_result(train_ID,data,test_seq):

    allevent,new_event,alls_with_index,length = get_dict_(data,'1S41')
    idx2char = np.array(allevent)
    char2idx={char:idx for idx,char in enumerate(allevent)}
    all_sentence=[]
    for i in alls_with_index:
        all_sentence.append(i[0])

    vocab_size = len(allevent)
    if int(length/200)*8 ==0:
        bs=8
    else:
        bs=int(length/200)*8

    batch_size = bs
    embedding_dim = 256
    rnn_units = 1024
    def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
        model = keras.models.Sequential([
          keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape = [batch_size, None]),
          keras.layers.LSTM(units = rnn_units,
                            stateful = True,
                            recurrent_initializer = 'glorot_uniform',
                            return_sequences = True),
          keras.layers.Dense(vocab_size),
      ])
        return model


    model_lstm = build_model(vocab_size,
                embedding_dim,
                rnn_units,
                batch_size = 1)
    model_lstm.load_weights('LSTM/'+train_ID+'_lstm_model.h5')
    model_lstm.build(tf.TensorShape([1, None]))

    predpr=pred_pr(model_lstm, test_seq,char2idx,idx2char,num_generate = 1)[0]
    return predpr

In [ ]:
#XGboost pre-processing

In [33]:
def get_xg_test_data(data,train_ID):
    data1=data[data['Train_ID']=='1S41']
    data1.index=range(0,len(data1))

    new_ID=[]
    ID_un = data1['ID'].unique().tolist()
    for i in ID_un:
        if i[0]=='B':
            new_ID.append(i[1:5])
        elif i[0:3]=='RDY' :
            new_ID.append('0'+i[3:6])
        elif i[0]=='R':
            new_ID.append(i[3:7])
        elif i[0:3]=='SDY':
            new_ID.append('0'+i[3:6])
        elif i[0]=='S':
            new_ID.append(i[3:7])
        new_ID_un=[]
    for i in new_ID:
        if i not in new_ID_un:
            new_ID_un.append(i)

    new_ID_un.sort()

    for j in new_ID_un:
        data1[j]=0

    data1[train_ID]=1
    data1.drop(['Train_ID'],inplace=True,axis=1)

    return data1

In [35]:
def get_ydata(train_ID,data):
    train1=data[data['Train_ID']==train_ID]
    train1['Timestamp']=range(1,len(train1)+1)

    def get_unique_pr_with0(train_ID,data):
    train1=data[data['Train_ID']==train_ID]
    new_ID=get_ID_withroute(train1['ID'].tolist()) 

    type1= train1['Type'].tolist()
    pr_type=[]

    for i in range(0,len(type1)):
        if type1[i]=='Panel Request':
            pr_type.append(new_ID[i])
        else:
            pr_type.append(0)

    pr_unique=[]
    for j in pr_type:
        if (j != 0)and (j not in pr_unique):
            pr_unique.append(j)


    return pr_unique,pr_type

    def get_ID_withroute(ID_data):
        new_ID=[]
        for i in ID_data:
            if i[0]=='B':
                new_ID.append(i[1:5])
            elif i[0:3]=='RDY' :
                new_ID.append(i)
            elif i[0]=='R':
                new_ID.append(i)
            elif i[0:3]=='SDY':
                new_ID.append('0'+i[3:6])
            elif i[0]=='S':
                new_ID.append(i[3:7])
        return new_ID  

    
    train1.drop(['State','Panel_Request','From_Berth','To_Berth'],inplace=True,axis=1)

    pr_unique,pr_type=get_unique_pr_with0(train_ID,data)

    pr_type_list=[]
    each=[]
    for i in pr_unique:
        each=[i,pr_unique.index(i)+1]
        pr_type_list.append(each)   
    return pr_type_list

In [ ]:
# Get XGBOOST predictions results
# Joint the predictions of lstm and xgboost for the final result

In [34]:
def get_xgb_result(train_ID,data,feature_change,index_test):
    data1=get_xg_test_data(data,train_ID)
    pd_testdata=pd.DataFrame(data1.iloc[index_test])
    testdata_ = pd_testdata.T
    testdata_=testdata_.iloc[:,8:]



    testcol=testdata_.columns.tolist()
    testcol

    for i in range(0,len(testcol)):
        if testcol[i][0]=='S':
            lbl = preprocessing.LabelEncoder()
            testdata_[testcol[i]] = lbl.fit_transform(testdata_[testcol[i]].astype(int))
        else:
            lbl = preprocessing.LabelEncoder()
            testdata_[testcol[i]] = lbl.fit_transform(testdata_[testcol[i]].astype(float))

    testdata_[feature_change].replace(testdata_[feature_change][index_test],1,inplace=True)
    loaded_model = joblib.load('save/'+train_ID+'_real_pr.pkl')
    ypred = loaded_model.predict(testdata_)

    pr_type_list=get_ydata(train_ID,data)

    for i in pr_type_list:
        if i[1]==ypred[0]:
            xgb_result=i[0]

    return xgb_result

In [ ]:
# Wrapping the entire prediction process into functions

In [54]:
def get_result(train_ID,data,test_seq,index_test_seq):
    position_pr=get_lstm_result(train_ID,data,test_seq)           # Gets the results of the LSTM
    if position_pr[0]=='R':                         # If the LSTM predicts that Panel_Request will follow, call XgBoost to predict it 
        feature_change=position_pr[3:7]                   # Get LSTM results as features
        index_test=index_test_seq[-1]                     # Use the last data as a location marker to issue PR
        result = get_xgb_result(train_ID,data,feature_change,index_test)  # Get the final prediction results
        return result
    else:
        print('No panel requests are required')             # if Panel_Request is not required. Print this sentence

In [ ]:
'''
2.An example of a prediction is provided for train 1S41.
'''

In [ ]:
# Two examples of predictions

In [36]:
data = pd.read_csv('full_data_with_state.csv',index_col= 0)

In [58]:
#The test sequence here is actually the event in the data, but only the event of the CA data left behind
test_seq1=['CAB0586BC833',
 'CABC833BDMLS',
 'CABAPUTBB305',
 'RDW5328',
 'CABB305B5332']

index_test_seq1=[34,35,36,37,39]

In [59]:
result= get_result('1S41',data,test_seq1,index_test_seq1)
result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

'RDW5324A(M)'

In [60]:
test_seq2=['CAB0574B0584',
 'CAB0584B0586',
 'CAB0586BC833',
 'CABC833BDMLS',
 'CABAPUTBB305',
 'RDW5328',
 'CABB305B5332',
 'RDW5324',
 'CAB5332B5330']

index_test_seq2=[32,33,34,35,36,37,39,40,43]

In [61]:
result= get_result('1S41',data,test_seq2,index_test_seq2)
result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

'RDW5316B(M)'